### Import Modules and Dependencies ###

In [1]:
import pandas as pd
import numpy as np

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
import requests

import json
import requests
from pprint import pprint

from api_file import api_key

### Read CSV File and Create a Dataframe of Thirty Cities ###

In [2]:
cities_list_df = pd.read_csv("CensusDataCities.csv")

cities_list_df.head()

,Ranking,Metro_Area,City,Metropollitan_Population
0,1,"New York-Newark, NY-NJ-CT-PA","New York, NY","23,689,255"
1,2,"Los Angeles-Long Beach, CA","Los Angeles, CA","18,688,022"
2,3,"Chicago-Naperville, IL-IN-WI","Chicago, IL","9,882,634"
3,4,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA","Washington, CD","9,665,892"
4,5,"San Jose-San Francisco-Oakland, CA","San Francisco, CA","8,751,807"


### Create Lists of Cities and States for Possible Input in API Call ###

In [3]:
cities = []

for city in cities_list_df["City"].str.split(",").str[0]:
    cities.append(city)
    
print(cities)

['New York', 'Los Angeles', 'Chicago', 'Washington', 'San Francisco', 'Boston', 'Dallas', 'Philadelphia', 'Houston', 'Miami', 'Atlanta', 'Detroit', 'Seattle', 'Minneapolis', 'Cleveland', 'Denver', 'Orlando', 'Portland', 'St. Louis', 'Pittsburgh', 'Charlotte', 'Sacramento', 'Salt Lake City', 'Kansas City', 'Columbus', 'Las Vegas', 'Indianapolis', 'Cincinnati', 'Raleigh', 'Milwaukee']


In [4]:
states = []

for state in cities_list_df["City"].str.split(", ").str[1]:
    states.append(state)
    
print(states)

['NY', 'CA', 'IL', 'CD', 'CA', 'MA', 'TX', 'PA', 'TX', 'FL', 'GA', 'MI', 'WA', 'MN', 'OH', 'CO', 'FL', 'OR', 'MO', 'PA', 'NC', 'CA', 'UT', 'MO', 'OH', 'NV', 'IN', 'OH', 'NC', 'WI']


### Use Beautiful Soup to Scrape an HTML Document with State and FIPS Codes ###

In [5]:
req = Request("https://www.mcc.co.mercer.pa.us/dps/state_fips_code_listing.htm", headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

webpage

b'<html>\r\n\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=windows-1252">\r\n<meta name="GENERATOR" content="Microsoft FrontPage 4.0">\r\n<meta name="ProgId" content="FrontPage.Editor.Document">\r\n<title>State FIPS Code Listing</title>\r\n</head>\r\n\r\n<body>\r\n\r\n<p align="center"><br>\r\n<b><font size="+3">State FIPS Code Listing</font></b>\r\n\r\n<p align="center"><img border="0" src="images/state_6.gif" width="733" height="172">\r\n<hr>\r\n<h3 align="center">Click on the highlighted State abbreviation for a list of\r\nCounties and County FIPS codes.</h3>\r\n<div align="center">\r\n  <center>\r\n<table border="1" width="687">\r\n  <tbody>\r\n    <tr>\r\n      <td align="center" width="85"><strong>State Abbreviation</strong></td>\r\n      <td align="center" width="46"><strong>FIPS Code</strong></td>\r\n      <td align="center" width="182"><strong>State Name</strong></td>\r\n      <td align="center" width="86"><strong>State Abbreviation</strong></td>\r\n

In [6]:
soup = bs(webpage, "html.parser")
type(soup)

bs4.BeautifulSoup

### Use Beautiful Soup Object to Create an HTML File for Conversion to a Dataframe ###

In [7]:
with open("StateFIPSCodes.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

In [8]:
state_fips = pd.read_html("StateFIPSCodes.html")[0]

state_fips

,0,1,2,3,4,5
0,State Abbreviation,FIPS Code,State Name,State Abbreviation,FIPS Code,State Name
1,AK,02,ALASKA,MS,28,MISSISSIPPI
2,AL,01,ALABAMA,MT,30,MONTANA
3,AR,05,ARKANSAS,NC,37,NORTH CAROLINA
4,AS,60,AMERICAN SAMOA,ND,38,NORTH DAKOTA
5,AZ,04,ARIZONA,NE,31,NEBRASKA
6,CA,06,CALIFORNIA,NH,33,NEW HAMPSHIRE
7,CO,08,COLORADO,NJ,34,NEW JERSEY
8,CT,09,CONNECTICUT,NM,35,NEW MEXICO
9,DC,11,DISTRICT OF COLUMBIA,NV,32,NEVADA


### Conversion of Two Sets of Columns from Dataframe into One Set ###

In [9]:
state_fips_df1 = state_fips.drop(state_fips.columns[[3, 4, 5]], axis=1)

state_fips_df1

,0,1,2
0,State Abbreviation,FIPS Code,State Name
1,AK,02,ALASKA
2,AL,01,ALABAMA
3,AR,05,ARKANSAS
4,AS,60,AMERICAN SAMOA
5,AZ,04,ARIZONA
6,CA,06,CALIFORNIA
7,CO,08,COLORADO
8,CT,09,CONNECTICUT
9,DC,11,DISTRICT OF COLUMBIA


In [10]:
state_fips_df1a = state_fips_df1.drop(state_fips_df1.index[28])

state_fips_df1a

,0,1,2
0,State Abbreviation,FIPS Code,State Name
1,AK,02,ALASKA
2,AL,01,ALABAMA
3,AR,05,ARKANSAS
4,AS,60,AMERICAN SAMOA
5,AZ,04,ARIZONA
6,CA,06,CALIFORNIA
7,CO,08,COLORADO
8,CT,09,CONNECTICUT
9,DC,11,DISTRICT OF COLUMBIA


In [11]:
state_fips_df1b = state_fips_df1a.drop(state_fips_df1a.index[0])

state_fips_df1b

,0,1,2
1,AK,02,ALASKA
2,AL,01,ALABAMA
3,AR,05,ARKANSAS
4,AS,60,AMERICAN SAMOA
5,AZ,04,ARIZONA
6,CA,06,CALIFORNIA
7,CO,08,COLORADO
8,CT,09,CONNECTICUT
9,DC,11,DISTRICT OF COLUMBIA
10,DE,10,DELAWARE


In [12]:
state_fips_df2 = state_fips.drop(state_fips.columns[[0, 1, 2]], axis=1)

state_fips_df2

,3,4,5
0,State Abbreviation,FIPS Code,State Name
1,MS,28,MISSISSIPPI
2,MT,30,MONTANA
3,NC,37,NORTH CAROLINA
4,ND,38,NORTH DAKOTA
5,NE,31,NEBRASKA
6,NH,33,NEW HAMPSHIRE
7,NJ,34,NEW JERSEY
8,NM,35,NEW MEXICO
9,NV,32,NEVADA


In [13]:
state_fips_df2a = state_fips_df2.drop(state_fips_df2.index[0])

state_fips_df2a

,3,4,5
1,MS,28,MISSISSIPPI
2,MT,30,MONTANA
3,NC,37,NORTH CAROLINA
4,ND,38,NORTH DAKOTA
5,NE,31,NEBRASKA
6,NH,33,NEW HAMPSHIRE
7,NJ,34,NEW JERSEY
8,NM,35,NEW MEXICO
9,NV,32,NEVADA
10,NY,36,NEW YORK


In [14]:
all_dfs = [state_fips_df1b, state_fips_df2a]

for df in all_dfs:
    df.columns = ["State Abbreviation", "FIPS Code", "State Name"]

state_fips_df = pd.concat(all_dfs).reset_index(drop=True)

state_fips_df

,State Abbreviation,FIPS Code,State Name
0,AK,02,ALASKA
1,AL,01,ALABAMA
2,AR,05,ARKANSAS
3,AS,60,AMERICAN SAMOA
4,AZ,04,ARIZONA
5,CA,06,CALIFORNIA
6,CO,08,COLORADO
7,CT,09,CONNECTICUT
8,DC,11,DISTRICT OF COLUMBIA
9,DE,10,DELAWARE


### Create a List of State FIPS Codes for Use in an API Call ###

In [15]:
state_fips_select = state_fips_df[state_fips_df["State Abbreviation"].isin(states)]["FIPS Code"]

print(state_fips_select)

5     06
6     08
10    12
11    13
16    17
17    18
21    25
24    26
25    27
26    29
29    37
35    32
36    36
37    39
39    41
40    42
46    48
47    49
51    53
52    55
Name: FIPS Code, dtype: object


### Read CSV File and Create a List of State FIPS Codes for Use in an API Call ###

In [22]:
county_fips_df = pd.read_csv("CountyFIPSCodes.csv")

county_fips_df.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


In [23]:
county_fips_select = []

for county in county_fips_df["County Code (FIPS)"]:
    county_fips_select.append(county)

print(county_fips_select)

[0, 0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### API Call to Census.gov ###

In [24]:
#Census Data API User Guide (Link): https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
#American Community Survey Data Variables (Link): https://api.census.gov/data/2013/acs/acs1/variables.html
#American Community Survey Data Variables (Link): https://api.census.gov/data/2018/acs/acs1/spp/variables.html
#American Community Survey Data Variables (Link): https://api.census.gov/data/1986/cbp/variables.html
#"https://api.census.gov/data/2018/acs/acs1?&get=NAME,B20005I_048E&for=state:*&key=your key here")

state_response = []

for state in state_fips_select:
    url = ("https://api.census.gov/data/2018/acs/acs1?&get=NAME,B25126_005E&for=county:*&for=state:*&key=ed92c098a31888a1903b422980d7d1646b3927ea")
    state_fips_data = requests.get(url).json()
    state_response.append(state_fips_data)

print(state_response)

[[['NAME', 'B25126_005E', 'state', 'county'], ['Newport County, Rhode Island', '0', '44', '005'], ['Washington County, Rhode Island', '140', '44', '009'], ['Oconee County, South Carolina', '63', '45', '073'], ['Guadalupe County, Texas', '820', '48', '187'], ['Snohomish County, Washington', '1526', '53', '061'], ['Ponce Municipio, Puerto Rico', '0', '72', '113'], ['Santa Clara County, California', '1026', '06', '085'], ['Stanislaus County, California', '211', '06', '099'], ['Palm Beach County, Florida', '832', '12', '099'], ['Forsyth County, Georgia', '648', '13', '117'], ['Sedgwick County, Kansas', '467', '20', '173'], ['Allegan County, Michigan', '85', '26', '005'], ['Kent County, Michigan', '733', '26', '081'], ['St. Louis County, Missouri', '81', '29', '189'], ['Camden County, New Jersey', '152', '34', '007'], ['Otero County, New Mexico', None, '35', '035'], ['Surry County, North Carolina', '84', '37', '171'], ['Rutherford County, Tennessee', '878', '47', '149'], ['Gregg County, Tex

In [25]:
pprint(state_response)

[[['NAME', 'B25126_005E', 'state', 'county'],
  ['Newport County, Rhode Island', '0', '44', '005'],
  ['Washington County, Rhode Island', '140', '44', '009'],
  ['Oconee County, South Carolina', '63', '45', '073'],
  ['Guadalupe County, Texas', '820', '48', '187'],
  ['Snohomish County, Washington', '1526', '53', '061'],
  ['Ponce Municipio, Puerto Rico', '0', '72', '113'],
  ['Santa Clara County, California', '1026', '06', '085'],
  ['Stanislaus County, California', '211', '06', '099'],
  ['Palm Beach County, Florida', '832', '12', '099'],
  ['Forsyth County, Georgia', '648', '13', '117'],
  ['Sedgwick County, Kansas', '467', '20', '173'],
  ['Allegan County, Michigan', '85', '26', '005'],
  ['Kent County, Michigan', '733', '26', '081'],
  ['St. Louis County, Missouri', '81', '29', '189'],
  ['Camden County, New Jersey', '152', '34', '007'],
  ['Otero County, New Mexico', None, '35', '035'],
  ['Surry County, North Carolina', '84', '37', '171'],
  ['Rutherford County, Tennessee', '878

  ['Augusta County, Virginia', None, '51', '015'],
  ['Chesapeake city, Virginia', '683', '51', '550'],
  ['Roanoke city, Virginia', '0', '51', '770'],
  ['Cabell County, West Virginia', '144', '54', '011'],
  ['Maricopa County, Arizona', '3644', '04', '013'],
  ['Jefferson County, Arkansas', '87', '05', '069'],
  ['Broomfield County, Colorado', '265', '08', '014'],
  ['Gallatin County, Montana', '130', '30', '031'],
  ['Butler County, Pennsylvania', '100', '42', '019'],
  ['Luzerne County, Pennsylvania', '506', '42', '079'],
  ['Kershaw County, South Carolina', None, '45', '055'],
  ['Loudoun County, Virginia', '1719', '51', '107'],
  ['Toa Alta Municipio, Puerto Rico', None, '72', '135'],
  ['Adams County, Colorado', '625', '08', '001'],
  ['Lafourche Parish, Louisiana', '613', '22', '057'],
  ['Rensselaer County, New York', '0', '36', '083'],
  ['Davidson County, North Carolina', '0', '37', '057'],
  ['Licking County, Ohio', '451', '39', '089'],
  ['Jackson County, Oregon', '14', '4

  ['Lonoke County, Arkansas', '363', '05', '085'],
  ['Carroll County, Georgia', '0', '13', '045'],
  ['Hall County, Georgia', '278', '13', '139'],
  ['Kosciusko County, Indiana', '182', '18', '085'],
  ['Vanderburgh County, Indiana', '44', '18', '163'],
  ['Hunterdon County, New Jersey', '36', '34', '019'],
  ['Niagara County, New York', '39', '36', '063'],
  ['Putnam County, New York', None, '36', '079'],
  ['Randolph County, North Carolina', '451', '37', '151'],
  ['Oklahoma County, Oklahoma', '1309', '40', '109'],
  ['Jefferson County, Texas', '238', '48', '245'],
  ['Marathon County, Wisconsin', '51', '55', '073'],
  ['Navajo County, Arizona', '19', '04', '017'],
  ['Tulare County, California', '515', '06', '107'],
  ['Mesa County, Colorado', '323', '08', '077'],
  ['Rapides Parish, Louisiana', '175', '22', '079'],
  ['St. Clair County, Michigan', '173', '26', '147'],
  ['Livingston County, New York', None, '36', '051'],
  ['Suffolk County, New York', '213', '36', '103'],
  ['Clev

  ['Montgomery County, Maryland', '664', '24', '031'],
  ['St. Louis city, Missouri', '0', '29', '510'],
  ['Essex County, New Jersey', '0', '34', '013'],
  ['Warren County, New York', '0', '36', '113'],
  ['Jefferson County, Ohio', '0', '39', '081'],
  ['Suffolk city, Virginia', '272', '51', '800'],
  ['King County, Washington', '1820', '53', '033'],
  ['Harrison County, West Virginia', '103', '54', '033'],
  ['Matanuska-Susitna Borough, Alaska', '132', '02', '170'],
  ['Middlesex County, Connecticut', '0', '09', '007'],
  ['Columbia County, Florida', None, '12', '023'],
  ['Rockdale County, Georgia', None, '13', '247'],
  ['Bannock County, Idaho', '10', '16', '005'],
  ['Flathead County, Montana', '88', '30', '029'],
  ['Bernalillo County, New Mexico', '458', '35', '001'],
  ['Brunswick County, North Carolina', '347', '37', '019'],
  ['Nash County, North Carolina', '0', '37', '127'],
  ['Wilkes County, North Carolina', None, '37', '193'],
  ['Hamilton County, Ohio', '60', '39', '061'

  ['Salem County, New Jersey', '119', '34', '033'],
  ['Queens County, New York', '471', '36', '081'],
  ['Allegheny County, Pennsylvania', '1042', '42', '003'],
  ['Hunt County, Texas', '261', '48', '231'],
  ['New London County, Connecticut', '226', '09', '011'],
  ['Sangamon County, Illinois', '0', '17', '167'],
  ['Bossier Parish, Louisiana', '527', '22', '015'],
  ['Calhoun County, Michigan', '0', '26', '025'],
  ['Muskingum County, Ohio', '0', '39', '119'],
  ['Bucks County, Pennsylvania', '515', '42', '017'],
  ['Orangeburg County, South Carolina', '6', '45', '075'],
  ['Bexar County, Texas', '3816', '48', '029'],
  ['Clallam County, Washington', '0', '53', '009'],
  ['Douglas County, Georgia', None, '13', '097'],
  ['Canyon County, Idaho', '296', '16', '027'],
  ['Clark County, Indiana', '155', '18', '019'],
  ['York County, Maine', '255', '23', '031'],
  ['Howard County, Maryland', '182', '24', '027'],
  ['Hennepin County, Minnesota', '449', '27', '053'],
  ['Jones County, Mis

  ['Christian County, Kentucky', '206', '21', '047'],
  ['Franklin County, Massachusetts', '0', '25', '011'],
  ['Hampden County, Massachusetts', '0', '25', '013'],
  ['Missoula County, Montana', '811', '30', '063'],
  ['Hancock County, Ohio', '395', '39', '063'],
  ['Stark County, Ohio', '23', '39', '151'],
  ['Harris County, Texas', '6852', '48', '201'],
  ['Roanoke County, Virginia', '70', '51', '161'],
  ['San Bernardino County, California', '854', '06', '071'],
  ['Bay County, Florida', '112', '12', '005'],
  ['Bartow County, Georgia', None, '13', '015'],
  ['Albany County, New York', '153', '36', '001'],
  ['Miami County, Ohio', '257', '39', '109'],
  ['Scioto County, Ohio', '0', '39', '145'],
  ['Pickens County, South Carolina', '246', '45', '077'],
  ['Yuba County, California', '121', '06', '115'],
  ['Barrow County, Georgia', '336', '13', '013'],
  ['Jackson County, Georgia', None, '13', '157'],
  ['Richmond County, Georgia', '56', '13', '245'],
  ['Troup County, Georgia', Non

  ['Portage County, Ohio', '286', '39', '133'],
  ['Marion County, Oregon', '57', '41', '047'],
  ['Centre County, Pennsylvania', '13', '42', '027'],
  ['Laurens County, South Carolina', '39', '45', '059'],
  ['Anderson County, Tennessee', '0', '47', '001'],
  ['Hanover County, Virginia', '152', '51', '085'],
  ['Kitsap County, Washington', '292', '53', '035'],
  ['Outagamie County, Wisconsin', '158', '55', '087'],
  ['Santa Cruz County, California', '0', '06', '087'],
  ['Bibb County, Georgia', '0', '13', '021'],
  ['Ada County, Idaho', '877', '16', '001'],
  ['Tazewell County, Illinois', '0', '17', '179'],
  ['Johnson County, Indiana', '0', '18', '081'],
  ['Johnson County, Iowa', '710', '19', '103'],
  ['Macomb County, Michigan', '410', '26', '099'],
  ['Rice County, Minnesota', '150', '27', '131'],
  ['Strafford County, New Hampshire', '97', '33', '017'],
  ['Onondaga County, New York', '0', '36', '067'],
  ['Douglas County, Oregon', '119', '41', '019'],
  ['Franklin County, Pennsy

  ['Clay County, Missouri', '359', '29', '047'],
  ['Yellowstone County, Montana', '382', '30', '111'],
  ['Jefferson County, New York', '44', '36', '045'],
  ['Armstrong County, Pennsylvania', '73', '42', '005'],
  ['Ellis County, Texas', '676', '48', '139'],
  ['Liberty County, Texas', None, '48', '291'],
  ['Webb County, Texas', '569', '48', '479'],
  ['James City County, Virginia', None, '51', '095'],
  ['Kenosha County, Wisconsin', '19', '55', '059'],
  ['Craighead County, Arkansas', '233', '05', '031'],
  ['Butler County, Kansas', '71', '20', '015'],
  ['Leavenworth County, Kansas', '116', '20', '103'],
  ['Riley County, Kansas', '181', '20', '161'],
  ['Hampshire County, Massachusetts', '100', '25', '015'],
  ['Cape Girardeau County, Missouri', '297', '29', '031'],
  ['Erie County, New York', '218', '36', '029'],
  ['Cleveland County, North Carolina', '104', '37', '045'],
  ['Clackamas County, Oregon', '8', '41', '005'],
  ['Sullivan County, Tennessee', '48', '47', '163'],
  ['G